# LLM Manager Real Streaming HelloWorld Demonstration

This notebook demonstrates the **LLM Manager's new real streaming functionality** using the MessageBuilder and actual AWS Bedrock `converse_stream` API to provide true real-time streaming responses.

## Key Features

- 🚀 **Real Streaming**: Uses actual AWS Bedrock `converse_stream` API with real-time output display
- 🔧 **MessageBuilder Integration**: Uses MessageBuilder for all message construction
- 📁 **Multi-Modal Support**: Streaming with images, documents, and text content
- 🔄 **Stream Recovery**: Intelligent retry with partial content preservation
- 📊 **Rich Metadata**: Complete streaming metrics, token usage, and performance data
- ⚡ **Real-Time Display**: See content being generated chunk by chunk
- 🛡️ **Error Handling**: Comprehensive stream interruption detection and recovery

## What's New

**Before (Placeholder Implementation):**
```python
# Used synchronous converse() call internally
streaming_response = manager.converse_stream(messages)
# Showed only final result
```

**After (Real Streaming Implementation):**
```python
# Uses actual AWS Bedrock converse_stream API with EventStream processing
streaming_response = manager.converse_stream(messages)
# Shows real-time streaming with chunk-by-chunk display
```

## Setup and Imports

In [1]:
import sys
import json
import time
from pathlib import Path
import logging
from datetime import datetime
from typing import Iterator, Any

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import the LLMManager and related classes
from bestehorn_llmmanager.llm_manager import LLMManager
from bestehorn_llmmanager.bedrock.models.llm_manager_structures import AuthConfig, RetryConfig, AuthenticationType, RetryStrategy
from bestehorn_llmmanager.bedrock.exceptions.llm_manager_exceptions import LLMManagerError, ConfigurationError, AuthenticationError

# Import MessageBuilder components (following established pattern)
from bestehorn_llmmanager import create_user_message, create_assistant_message, create_message
from bestehorn_llmmanager.message_builder_enums import RolesEnum, ImageFormatEnum, DocumentFormatEnum, VideoFormatEnum

# Import streaming components and display utilities
from bestehorn_llmmanager.bedrock.models.bedrock_response import StreamingResponse
from bestehorn_llmmanager.util.streaming_display import display_streaming_response, display_streaming_summary, display_recovery_information

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")
print("🚀 Real streaming functionality with MessageBuilder imported and ready!")

✅ Imports successful!
📁 Working directory: d:\Code Workspace\LLMManager\notebooks
🚀 Real streaming functionality with MessageBuilder imported and ready!


## Helper Functions

In [2]:
def display_file_info(file_path: str, content_type: str = "file"):
    """Display information about a file (following established pattern)."""
    path = Path(file_path)
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"📁 {content_type.title()}: {path.name}")
        print(f"   📏 Size: {size_mb:.2f} MB ({path.stat().st_size:,} bytes)")
        print(f"   📍 Path: {path}")
        return True
    else:
        print(f"❌ {content_type.title()} file not found: {file_path}")
        return False

print("✅ Helper functions defined!")

✅ Helper functions defined!


## Initialize the LLMManager

In [3]:
print("🚀 Initializing LLMManager...")

# Use known working models (refreshing data if needed)
models = ["Claude 3.5 Sonnet v2", "Claude 3 Haiku", "Claude 3 Sonnet"]
regions = ["us-east-1", "us-west-2", "eu-west-1"]

auth_config = AuthConfig(auth_type=AuthenticationType.PROFILE, profile_name="default")
retry_config = RetryConfig(max_retries=3, retry_strategy=RetryStrategy.REGION_FIRST)

try:
    manager = LLMManager(models=models, regions=regions, auth_config=auth_config, retry_config=retry_config, timeout=30)
    print(f"✅ LLMManager initialized successfully!")
    validation = manager.validate_configuration()
    print(f"   Valid: {'✅' if validation['valid'] else '❌'} {validation['valid']}")
    print(f"   Model/Region combinations: {validation['model_region_combinations']}")
except Exception as e:
    print(f"⚠️ Initial setup failed, refreshing model data...")
    # Refresh model data if needed
    from bestehorn_llmmanager.bedrock.UnifiedModelManager import UnifiedModelManager
    umm = UnifiedModelManager()
    umm.refresh_unified_data()
    print("✅ Model data refreshed")
    
    manager = LLMManager(models=models, regions=regions, auth_config=auth_config, retry_config=retry_config, timeout=30)
    print(f"✅ LLMManager initialized successfully after refresh!")
    validation = manager.validate_configuration()
    print(f"   Valid: {'✅' if validation['valid'] else '❌'} {validation['valid']}")
    print(f"   Model/Region combinations: {validation['model_region_combinations']}")

🚀 Initializing LLMManager...
✅ LLMManager initialized successfully!
   Valid: ✅ True
   Model/Region combinations: 8


## Example 1: Basic Real-Time Streaming with MessageBuilder 🚀

Demonstrating real streaming with MessageBuilder for message construction.

In [5]:
print("🚀 Example 1: Basic Real-Time Streaming with MessageBuilder")
print("=" * 60)

# Create message using MessageBuilder (following established pattern)
message = create_user_message() \
    .add_text("Write a short story about a robot learning to paint. Make it about 3 paragraphs and make it engaging.") \
    .build()

print(f"🔧 Built message using MessageBuilder with {len(message['content'])} content blocks")
print(f"📝 Prompt: {message['content'][0]['text'][:100]}...")

try:
    print("\n🌊 Starting real-time streaming...")
    streaming_response = manager.converse_stream(
        messages=[message], 
        inference_config={"maxTokens": 800, "temperature": 0.7}
    )
    
    print("\n📺 Real-time streaming output:")
    print("-" * 50)
    
    # Real streaming iteration - content appears as it arrives!
    try:
        for chunk in streaming_response:
            print(chunk, end='', flush=True)  # Real-time display!
    except Exception as stream_error:
        print(f"\n❌ Stream interrupted: {stream_error}")
    
    print(f"\n{'-' * 50}")
    print("✅ Streaming completed!")
    
    # Now show model/region info after streaming completes
    print(f"🤖 Model: {streaming_response.model_used}")
    print(f"🌍 Region: {streaming_response.region_used}")
    
    # Show final metadata (available after streaming completes)
    print(f"\n📊 Streaming Results:")
    print(f"   Success: {streaming_response.success}")
    print(f"   Total Duration: {streaming_response.total_duration_ms:.1f}ms" if streaming_response.total_duration_ms else "   Duration: N/A")
    print(f"   Content Parts: {len(streaming_response.content_parts)}")
    print(f"   Stop Reason: {streaming_response.stop_reason or 'N/A'}")
    
    # Token usage (available after completion)
    usage = streaming_response.get_usage()
    if usage:
        print(f"\n🎯 Token Usage:")
        print(f"   Input: {usage.get('input_tokens', 0)}, Output: {usage.get('output_tokens', 0)}, Total: {usage.get('total_tokens', 0)}")
    
except Exception as e:
    print(f"❌ Error in basic streaming: {e}")

🚀 Example 1: Basic Real-Time Streaming with MessageBuilder
🔧 Built message using MessageBuilder with 1 content blocks
📝 Prompt: Write a short story about a robot learning to paint. Make it about 3 paragraphs and make it engaging...

🌊 Starting real-time streaming...

📺 Real-time streaming output:
--------------------------------------------------
The Artbot's First Masterpiece

Unit-7 stared at the blank canvas with its optical sensors whirring in contemplation. After analyzing 10,457 classical paintings and studying countless hours of human artists at work, it still couldn't quite grasp why its own attempts felt so... mechanical. Its titanium fingers delicately gripped the brush, servos humming softly as it mixed colors on the palette. Despite its perfect precision and ability to replicate any image with photographic accuracy, something was missing.

One rainy afternoon, while observing a child finger-painting in the park, Unit-7 noticed something peculiar. The child wasn't following 

## Example 2: Multi-Modal Streaming with Local Image 🖼️

Using MessageBuilder with local images for streaming analysis and the new streaming display utility.

In [6]:
print("🖼️ Example 2: Multi-Modal Streaming with Local Image")
print("=" * 52)

# Use established image path from the existing notebook
eiffel_image_path = "../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg"

if display_file_info(eiffel_image_path, "image"):
    try:
        # Build message using MessageBuilder with local image (following established pattern)
        message = create_user_message() \
            .add_text("Please analyze this image in detail. Describe the architecture, setting, and notable features. Stream your analysis as you observe different aspects.") \
            .add_local_image(path_to_local_file=eiffel_image_path, max_size_mb=5.0) \
            .build()
        
        print(f"🔧 Built multi-modal message with {len(message['content'])} content blocks using MessageBuilder")
        print(f"   📸 Image format detected: {message['content'][1]['image']['format']}")
        
        print("\n🌊 Starting streaming image analysis...")
        streaming_response = manager.converse_stream(
            messages=[message], 
            inference_config={"maxTokens": 1000, "temperature": 0.4}
        )
        
        print("\n📺 Real-time streaming output:")
        print("-" * 50)
        
        # Actually consume the stream to trigger API calls
        try:
            for chunk in streaming_response:
                print(chunk, end='', flush=True)  # Real-time display!
        except Exception as stream_error:
            print(f"\n❌ Stream interrupted: {stream_error}")
        
        print(f"\n{'-' * 50}")
        
        # Now use the new streaming display utility after streaming completes
        display_streaming_response(
            streaming_response=streaming_response,
            title="🖼️ Image Analysis Streaming Response",
            show_content=False,  # We already showed it in real-time
            content_preview_length=200
        )
        
        # Show recovery information if available
        display_recovery_information(streaming_response=streaming_response)
        
    except Exception as e:
        print(f"❌ Error in image streaming: {e}")
else:
    print("⚠️ Skipping image example - file not found")

🖼️ Example 2: Multi-Modal Streaming with Local Image
📁 Image: 1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg
   📏 Size: 0.41 MB (429,550 bytes)
   📍 Path: ..\images\1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg
🔧 Built multi-modal message with 2 content blocks using MessageBuilder
   📸 Image format detected: jpeg

🌊 Starting streaming image analysis...

📺 Real-time streaming output:
--------------------------------------------------
Let me analyze this iconic image of the Eiffel Tower in Paris...

The photograph is taken on what appears to be a perfect summer day, with a deep blue, cloudless sky serving as a dramatic backdrop. The Eiffel Tower rises majestically from the Champ de Mars, its iron lattice work creating the distinctive silhouette that has become synonymous with Paris.

The tower's intricate structural details are clearly visible - the four curved pillars that merge into the tapering tower, the multiple levels of platforms, and the complex geometric patterns of the

## Example 3: Advanced Streaming with Display Utilities 🔧

Demonstrating the new streaming display utilities for comprehensive response analysis.

In [7]:
print("🔧 Example 3: Advanced Streaming with Display Utilities")
print("=" * 56)

# Create a more complex message for detailed analysis
message = create_user_message() \
    .add_text("Explain the concept of machine learning in detail, including supervised and unsupervised learning, neural networks, and real-world applications. Make it comprehensive but accessible.") \
    .build()

print(f"🔧 Built comprehensive prompt with {len(message['content'])} content blocks")

try:
    print("\n🌊 Starting advanced streaming with display utilities...")
    streaming_response = manager.converse_stream(
        messages=[message], 
        inference_config={"maxTokens": 1500, "temperature": 0.6}
    )
    
    print("\n📺 Real-time streaming output:")
    print("-" * 50)
    
    # Stream the content in real-time
    try:
        for chunk in streaming_response:
            print(chunk, end='', flush=True)
    except Exception as stream_error:
        print(f"\n❌ Stream interrupted: {stream_error}")
    
    print(f"\n{'-' * 50}")
    
    # Use the comprehensive display utility
    display_streaming_response(
        streaming_response=streaming_response,
        title="🔧 Advanced Streaming Response Analysis",
        show_content=False,  # Already displayed in real-time
        show_metadata=True,
        show_timing=True,
        show_usage=True,
        show_errors=True
    )
    
    # Show a concise summary
    display_streaming_summary(
        streaming_response=streaming_response,
        title="📊 Streaming Performance Summary"
    )
    
except Exception as e:
    print(f"❌ Error in advanced streaming: {e}")

🔧 Example 3: Advanced Streaming with Display Utilities
🔧 Built comprehensive prompt with 1 content blocks

🌊 Starting advanced streaming with display utilities...

📺 Real-time streaming output:
--------------------------------------------------
Here's a comprehensive explanation of machine learning:

Machine Learning Fundamentals:
Machine learning (ML) is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. The core idea is that systems can identify patterns in data and make decisions with minimal human intervention.

Types of Machine Learning:

1. Supervised Learning:
- Works with labeled data
- Algorithm learns from known input-output pairs
- Examples:
  * Classification (predicting categories)
  * Regression (predicting continuous values)
- Common applications:
  * Spam detection
  * Image recognition
  * Price prediction

2. Unsupervised Learning:
- Works with unlabeled data
- Finds hidden patterns/struct